## Installing default packages

In [19]:
using Pkg;

# packages to install, first time can take a while downloading, please be patient ...
packages=["HTTP","JSON","DataFrames","Plots","StatPlots","PlotThemes","GR","PyPlot","PyCall","LaTeXStrings","Plotly","PlotlyJS"]

for package in packages
    if get(Pkg.installed(),package,-1) == -1
        println(" getting package : ", package)
        Pkg.add(package)
    end
end

## Creating generic functions for Binance API data retrieval

In [21]:
# show all data for a given value
function showAll(value)
    show(IOContext(stdout, :displaysize => (10^6, 10^6)), "text/plain", value)
end

using HTTP, JSON, Dates
import Printf.@sprintf

# base URL of the Binance API
const BINANCE_API_REST = "https://api.binance.com/api/v1/ticker/";

# function HTTP response 2 JSON
function r2j(response)
    JSON.parse(String(response))
end

# retrieve data from binance for "allPrices", "24hr", "allBookTickers"
function getBinanceDataFor(withSubject)
    r = HTTP.request("GET", string(BINANCE_API_REST, withSubject))
    r2j(r.body)
end

# current Binance market
function getBinanceMarket()
    r = HTTP.request("GET", "https://www.binance.com/exchange/public/product")
    r2j(r.body)["data"]
end

# binance get candlesticks/klines data
function getBinanceKline(symbol, startDateTime, endDateTime ; interval="1m")
    startTime = @sprintf("%.0d",Dates.datetime2unix(startDateTime) * 1000)
    endTime = @sprintf("%.0d",Dates.datetime2unix(endDateTime) * 1000)
    query = string("symbol=", symbol, "&interval=", interval, "&startTime=", startTime, "&endTime=", endTime)
    r = HTTP.request("GET", string("https://api.binance.com/api/v1/klines?",  query))
    r2j(r.body)
end

function getBinanceKlineDataframe(symbol, startDateTime, endDateTime ; interval="1m")
    klines = getBinanceKline(symbol, startDateTime, endDateTime; interval= interval)
    result = hcat(map(z -> map(x -> typeof(x) == String ? parse(Float64, x) : x, z), klines)...)';

    #println(size(result))
    if size(result,2) == 0
        return nothing
    end

    symbolColumnData = map(x -> symbol, collect(1:size(result, 1)));
    df = DataFrame([symbolColumnData, Dates.unix2datetime.(result[:,1]/1000) ,result[:,2],result[:,3],result[:,4],result[:,5],result[:,6],result[:,8],Dates.unix2datetime.(result[:,7] / 1000),result[:,9],result[:,10],result[:,11]], [:symbol,:startDate,:open,:high,:low,:close,:volume,:quoteAVolume, :endDate, :trades, :tbBaseAVolume,:tbQuoteAVolume]);
end

# getFloatValueArray
getFloatValueArray(withKey, withDictArr) = map(dict -> parse(Float32, dict[withKey]) ,withDictArr)
getStringValueArray(withKey, withDictArr) = map(dict -> convert(String, dict[withKey]) ,withDictArr)
# filter
filterOnRegex(matcher,withDictArr; withKey="symbol") = filter(x-> match(Regex(matcher), x[withKey]) != nothing, withDictArr);

## Retrieve Market Symbols ending with BTC

In [22]:
using DataFrames,Statistics;

# get 24H tickerdata of all assets
hr24 = getBinanceDataFor("24hr");
# previous post we determined that BTC currency was the most obvious currency to tade in, most avalable symbols, second highest USDT $ volume.
hr24BTC = filterOnRegex("BTC\$", hr24)

# all symbols ending with BTC
symbols = sort!(getStringValueArray("symbol", hr24BTC));

## Retrieve Data with interval 1 month "1M" and 1 minute "1m"

In [23]:
res = nothing;
# loop through all the symbols en retrieve kline data from binance api
current_datetime = now();
print("retrieving data from binance $current_datetime for :")

for symbol in symbols
    global res
    print(" $symbol")
    dfM = getBinanceKlineDataframe(symbol, now(UTC)-Month(6),now(); interval="1M")
    if(dfM == nothing)
        continue
    end

    # append most recent 5m data, if no trades occured skip this
    dfm = getBinanceKlineDataframe(symbol, now(UTC)-Minute(5),now(UTC); interval="1m")
    if(dfm == nothing)
        continue
    end

    append!(dfM,dfm)

    if res == nothing
        res = dfM;
    else
        append!(res,dfM)
    end
end

alldata = by(res, :symbol, df -> DataFrame(current = df[:close][end], min = minimum(df[:low]), changeMin = (df[:close][end] - minimum(df[:low]))/minimum(df[:low])*100, max =maximum(df[:high]), changeMax = (maximum(df[:high])-df[:close][end]) / maximum(df[:high])*100, trades =sum(df[:trades]),volume =sum(df[:volume])));


retrieving data from binance 2018-09-09T12:42:44.868 for : ADABTC ADXBTC AEBTC AGIBTC AIONBTC AMBBTC APPCBTC ARDRBTC ARKBTC ARNBTC ASTBTC BATBTC BCCBTC BCDBTC BCNBTC BCPTBTC BLZBTC BNBBTC BNTBTC BQXBTC BRDBTC BTGBTC BTSBTC CDTBTC CHATBTC CLOAKBTC CMTBTC CNDBTC CVCBTC DASHBTC DATABTC DENTBTC DGDBTC DLTBTC DNTBTC DOCKBTC EDOBTC ELFBTC ENGBTC ENJBTC EOSBTC ETCBTC ETHBTC EVXBTC FUELBTC FUNBTC GASBTC GNTBTC GRSBTC GTOBTC GVTBTC GXSBTC HCBTC HOTBTC HSRBTC ICNBTC ICXBTC INSBTC IOSTBTC IOTABTC IOTXBTC KEYBTC KMDBTC KNCBTC LENDBTC LINKBTC LOOMBTC LRCBTC LSKBTC LTCBTC LUNBTC MANABTC MCOBTC MDABTC MFTBTC MODBTC MTHBTC MTLBTC NANOBTC NASBTC NAVBTC NCASHBTC NEBLBTC NEOBTC NPXSBTC NULSBTC NXSBTC OAXBTC OMGBTC ONTBTC OSTBTC PHXBTC PIVXBTC POABTC POEBTC POLYBTC POWRBTC PPTBTC QKCBTC QLCBTC QSPBTC QTUMBTC RCNBTC RDNBTC REPBTC REQBTC RLCBTC RPXBTC SALTBTC SCBTC SKYBTC SNGLSBTC SNMBTC SNTBTC STEEMBTC STORJBTC STORMBTC STRATBTC SUBBTC SYSBTC THETABTC TNBBTC TNTBTC TRIGBTC TRXBTC TUSDBTC VENBTC VETBTC VIAB

## sort on volume/trades

In [24]:
sort!(alldata, [:volume,:trades]; rev=true);

## who is the loser ?

In [26]:
head(sort(alldata[1:40,:], [:changeMin]; rev=false))

,symbol,current,min,changeMin,max,changeMax,trades,volume
1,ADABTC,1.228e-5,1.201e-5,2.24813,4.19e-5,70.6921,10002842,2.03433e10
2,WPRBTC,2.68e-6,2.61e-6,2.68199,1.905e-5,85.9318,1806944,2.37861e9
3,LENDBTC,1.81e-6,1.76e-6,2.84091,1.086e-5,83.3333,1264368,3.55447e9
4,SNGLSBTC,2.92e-6,2.83e-6,3.18021,2.295e-5,87.2767,1184982,2.31718e9
5,FUNBTC,2.25e-6,2.17e-6,3.68664,6.51e-6,65.4378,1659178,6.09257e9
6,VIBBTC,4.81e-6,4.63e-6,3.88769,2.9e-5,83.4138,1143303,2.57617e9
